In [40]:
!pip install tqdm


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 78.5/78.5 kB 733.6 kB/s eta 0:00:00


In [18]:
import cv2
import mediapipe as mp

In [19]:
mp_drawing = mp.solutions.drawing_utils  # Drawing helpers
mp_pose = mp.solutions.pose  # Mediapipe Solutions


In [17]:
cap = cv2.VideoCapture(0)

#pose model
with mp_pose.Pose() as pose_tracker:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        result= pose_tracker.process(image)
        pose_landmarks = result.pose_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(
                                      color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(
                                      color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [19]:
(pose_landmarks.landmark)[0].visibility

0.9999074935913086

# Capture Landmarks and Export CSV

In [13]:
import csv
import os
import numpy as np
import tqdm

In [20]:
num_of_landmarks = len(pose_landmarks.landmark)
num_of_columns = 1 + num_of_landmarks

In [65]:
landmark_names = [
        'nose',
        'left_eye_inner', 'left_eye', 'left_eye_outer',
        'right_eye_inner', 'right_eye', 'right_eye_outer',
        'left_ear', 'right_ear',
        'mouth_left', 'mouth_right',
        'left_shoulder', 'right_shoulder',
        'left_elbow', 'right_elbow',
        'left_wrist', 'right_wrist',
        'left_pinky_1', 'right_pinky_1',
        'left_index_1', 'right_index_1',
        'left_thumb_2', 'right_thumb_2',
        'left_hip', 'right_hip',
        'left_knee', 'right_knee',
        'left_ankle', 'right_ankle',
        'left_heel', 'right_heel',
        'left_foot_index', 'right_foot_index',
    ]

In [71]:
landmarks = ['class']
for i, name in enumerate(landmark_names):
    landmarks += ['{}_x'.format(name), '{}_y'.format(name), '{}_z'.format(name), '{}_v'.format(name)]
len(landmarks)

133

In [72]:
with open('video_coordinate.csv', mode = 'w', newline ='') as f:
    csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

# Read data from folder

In [117]:
images_in_folder = 'yoga_poses_images_in'
images_out_folder = 'yoga_poses_images_out'

pose_class_names = [n for n in os.listdir(images_in_folder) if not n.startswith('.')]
for pose_class_name in pose_class_names:
    image_names = sorted([n for n in os.listdir(os.path.join(images_in_folder, pose_class_name)) 
                      if not n.startswith('.')])
    
    try:
        os.mkdir(os.path.join(images_out_folder, pose_class_name))
    except FileExistsError:
        pass
        
    for image_name in tqdm.tqdm(image_names, position=0):
        input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

      # Initialize fresh pose tracker and run it.
        with mp_pose.Pose() as pose_tracker:
            result = pose_tracker.process(image=input_frame)
            pose_landmarks = result.pose_landmarks
        
    #write the data to a csv
        if pose_landmarks is not None:
            pose = pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            pose_row.insert(0, pose_class_name)
        
            try:
                with open('coordinate.csv', mode = 'a', newline ='') as f:
                    csv_writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                    csv_writer.writerow(pose_row)
            except Exception as e:
                print("An error occured", e)
        
    # Save image with pose prediction (if pose was detected).
        output_frame = input_frame.copy()
        if pose_landmarks is not None:
            mp_drawing.draw_landmarks(
            image=output_frame,
            landmark_list=pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS)
        output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(images_out_folder, pose_class_name ,image_name), output_frame)

100%|████████████████████████████████████████████████████████████████████████████████| 153/153 [00:49<00:00,  3.08it/s]


# Read data from user camera and populate the csv

In [154]:
csv_file_name = "video.csv"

In [155]:
class_name = "t"

In [157]:
cap = cv2.VideoCapture(0)

#pose model
with mp_pose.Pose() as pose_tracker:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        result= pose_tracker.process(image)
        pose_landmarks = result.pose_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(
                                      color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(
                                      color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        
        try:
            pose = pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            pose_row.insert(0, class_name)  
            
            with open(csv_file_name, mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(pose_row) 
        except:
            pass           
        
        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection
No detection

# Add video data to CSV

In [73]:
videos_in_folder = 'yoga_poses_videos_in'
videos_out_folder = 'yoga_poses_videos_out'

In [74]:
pose_class_names = [n for n in os.listdir(videos_in_folder) if not n.startswith('.')]

print(pose_class_names)

for pose_class_name in pose_class_names:
    
    video_names = sorted([n for n in os.listdir(os.path.join(videos_in_folder, pose_class_name)) 
                      if not n.startswith('.')])
    

    
    try:
        os.mkdir(os.path.join(videos_out_folder, pose_class_name))
    except FileExistsError:
        pass
   
    for video_name in tqdm.tqdm(video_names, position=0):
        
        print(f"Processing {os.path.join(videos_in_folder, pose_class_name, video_name)}")
        
        video_cap = cv2.VideoCapture(os.path.join(videos_in_folder, pose_class_name, video_name))

        # Get some video parameters to generate output video with classificaiton.
        video_n_frames = video_cap.get(cv2.CAP_PROP_FRAME_COUNT)
        video_fps = video_cap.get(cv2.CAP_PROP_FPS)
        video_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        video_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        #out_video = cv2.VideoWriter(os.path.join(videos_out_folder, pose_class_name, video_name[:-4]+'.avi'), cv2.VideoWriter_fourcc(*'MJPG'), video_fps, (video_width, video_height))
        output_frame = None
        
        with mp_pose.Pose() as pose_tracker:
            while True:
    # Get next frame of the video.
                success, frame = video_cap.read()
                if not success:
                    break

    # Run pose tracker.
                input_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                input_frame.flags.writeable = False
        
                result = pose_tracker.process(image=input_frame)
        
                input_frame.flags.writeable = True
                pose_landmarks = result.pose_landmarks
        
                input_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
                
                try:
                    pose = pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                    pose_row.insert(0, pose_class_name)  
            
                    with open("video_coordinate.csv", mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        csv_writer.writerow(pose_row) 
                except:
                    pass

    # Draw pose prediction.
                #output_frame = input_frame.copy()
                #if pose_landmarks is not None:
                    #mp_drawing.draw_landmarks(
                    #image=output_frame,
                    #landmark_list=pose_landmarks,
                    #connections=mp_pose.POSE_CONNECTIONS)
            
                #out_video.write(np.array(output_frame))

['mountain', 't', 'tree']



  0%|                                                                                                                        | 0/5 [00:00<?, ?it/s]

Processing yoga_poses_videos_in\mountain\IMG_1104.mov



 20%|██████████████████████▍                                                                                         | 1/5 [00:03<00:15,  3.95s/it]

Processing yoga_poses_videos_in\mountain\IMG_1107.mov


...

  0%|                                                                                                                        | 0/2 [00:00<?, ?it/s]

Processing yoga_poses_videos_in\tree\IMG_1113.MOV



 50%|████████████████████████████████████████████████████████                                                        | 1/2 [00:15<00:15, 15.50s/it]

Processing yoga_poses_videos_in\tree\IMG_1114.MOV


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.95s/it]


In [38]:
video_path = "C:\Dev\Python\ML\MinorModel\yoga_poses_videos_in\mountain\IMG_1111.mov"

In [59]:
video_cap = cv2.VideoCapture(video_path)

video_n_frames = video_cap.get(cv2.CAP_PROP_FRAME_COUNT)
video_fps = video_cap.get(cv2.CAP_PROP_FPS)
video_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

ret, frame = video_cap.read()

out_video = cv2.VideoWriter("./yoga_poses_videos_out/test1.avi", cv2.VideoWriter_fourcc(*'MJPG'), video_fps, (video_width, video_height))
with mp_pose.Pose() as pose_tracker:
    
    while True:
        # Get next frame of the video.
        success, frame = video_cap.read()
        if not success:
            break
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        result= pose_tracker.process(image)
        pose_landmarks = result.pose_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        output_frame = image.copy()
        if pose_landmarks is not None:
            mp_drawing.draw_landmarks(
            image=output_frame,
            landmark_list=pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS)
            
        out_video.write(np.array(output_frame))




IndentationError: expected an indented block (3942386080.py, line 13)

# Starting on the Model

In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [76]:
df = pd.read_csv('video_coordinate.csv')

In [78]:
df.tail()

,class,nose_x,nose_y,nose_z,nose_v,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_inner_v,left_eye_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
2635,tree,0.540952,0.370157,-0.610867,0.998689,0.546606,0.360834,-0.570483,0.998560,0.551644,...,0.121898,0.866602,0.530067,0.646003,0.298245,0.596630,0.558066,0.826651,-0.064042,0.916052
2636,tree,0.541086,0.370236,-0.599057,0.998713,0.546762,0.360910,-0.559091,0.998583,0.551817,...,0.137327,0.869587,0.530633,0.652358,0.282312,0.601163,0.557937,0.826652,-0.043260,0.918196
2637,tree,0.541177,0.370315,-0.594462,0.998707,0.546808,0.360985,-0.554821,0.998577,0.551867,...,0.147485,0.871700,0.530562,0.652303,0.303158,0.603325,0.557845,0.826812,-0.032156,0.919582
2638,tree,0.541243,0.370426,-0.593205,0.998734,0.546700,0.361107,-0.553497,0.998608,0.551717,...,0.104934,0.874484,0.530495,0.652202,0.283407,0.607735,0.557820,0.827178,-0.078342,0.922081
2639,tree,0.541575,0.370545,-0.591754,0.998755,0.546825,0.361238,-0.552208,0.998632,0.551806,...,0.116404,0.877301,0.530491,0.651942,0.284691,0.610310,0.557799,0.827287,-0.062960,0.924173


In [79]:
#features
X = df.drop('class', axis = 1) 
#target
y = df['class'] 

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)

In [81]:
y_test

60      mountain
59      mountain
227     mountain
755     mountain
2220        tree
          ...   
864     mountain
720     mountain
1809           t
636     mountain
1265           t
Name: class, Length: 792, dtype: object

# ML pipeline

In [82]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

In [83]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rfc':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gbc':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [84]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [85]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rfc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gbc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

# Evaluate Model

In [86]:
fit_models['rfc'].predict(X_test)

array(['mountain', 'mountain', 'mountain', 'mountain', 'tree', 'tree',
       't', 't', 'tree', 'tree', 't', 'tree', 'tree', 'mountain',
       'mountain', 'mountain', 'mountain', 'tree', 'mountain', 't',
       'tree', 't', 'mountain', 't', 't', 'tree', 'tree', 'tree',
       'mountain', 'tree', 't', 'mountain', 'mountain', 't', 'mountain',
       'mountain', 'mountain', 't', 't', 'mountain', 'tree', 'mountain',
       'tree', 't', 'mountain', 'mountain', 'tree', 'tree', 'mountain',
       't', 'tree', 'tree', 'mountain', 'mountain', 'mountain', 'tree',
       't', 'mountain', 't', 'tree', 'mountain', 'mountain', 't',
       'mountain', 'mountain', 'mountain', 't', 'tree', 't', 't',
       'mountain', 'tree', 't', 'mountain', 't', 'mountain', 't', 't',
       'mountain', 't', 't', 'mountain', 'tree', 'mountain', 't',
       'mountain', 'tree', 'mountain', 'tree', 'tree', 't', 'mountain',
       'tree', 'mountain', 't', 't', 't', 'mountain', 't', 'mountain',
       't', 't', 'tree', 'm

In [87]:
from sklearn.metrics import accuracy_score
import pickle

In [88]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat), sep = ": ")

lr: 1.0
rc: 1.0
rfc: 1.0
gbc: 1.0


In [89]:
fit_models['rfc'].predict(X_test)

array(['mountain', 'mountain', 'mountain', 'mountain', 'tree', 'tree',
       't', 't', 'tree', 'tree', 't', 'tree', 'tree', 'mountain',
       'mountain', 'mountain', 'mountain', 'tree', 'mountain', 't',
       'tree', 't', 'mountain', 't', 't', 'tree', 'tree', 'tree',
       'mountain', 'tree', 't', 'mountain', 'mountain', 't', 'mountain',
       'mountain', 'mountain', 't', 't', 'mountain', 'tree', 'mountain',
       'tree', 't', 'mountain', 'mountain', 'tree', 'tree', 'mountain',
       't', 'tree', 'tree', 'mountain', 'mountain', 'mountain', 'tree',
       't', 'mountain', 't', 'tree', 'mountain', 'mountain', 't',
       'mountain', 'mountain', 'mountain', 't', 'tree', 't', 't',
       'mountain', 'tree', 't', 'mountain', 't', 'mountain', 't', 't',
       'mountain', 't', 't', 'mountain', 'tree', 'mountain', 't',
       'mountain', 'tree', 'mountain', 'tree', 'tree', 't', 'mountain',
       'tree', 'mountain', 't', 't', 't', 'mountain', 't', 'mountain',
       't', 't', 'tree', 'm

In [90]:
y_test

60      mountain
59      mountain
227     mountain
755     mountain
2220        tree
          ...   
864     mountain
720     mountain
1809           t
636     mountain
1265           t
Name: class, Length: 792, dtype: object

# Save Model

In [91]:
with open('mid_poses.pkl', 'wb') as f:
    pickle.dump(fit_models['rfc'], f)